In [2]:
import json
import requests
import pandas as pd
import numpy as np
r = requests.get('https://api.covidtracking.com/v1/states/daily.json')
read_states = r.json()
states_covid_daily = pd.DataFrame(read_states)
states_covid_daily=states_covid_daily[states_covid_daily['date']==20210307]

def clean_state_name(state_acroname):
        state= {
        'AL': 'Alabama',
        'AK': 'Alaska',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'AR': 'Arkansas',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DE': 'Delaware',
        'DC': 'District of Columbia',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'IA': 'Iowa',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'ME': 'Maine',
        'MD': 'Maryland',
        'MA': 'Massachusetts',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MS': 'Mississippi',
        'MO': 'Missouri',
        'MT': 'Montana',
        'NE': 'Nebraska',
        'NV': 'Nevada',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NY': 'New York',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'MP':'Northern Mariana Islands',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VT': 'Vermont',
        'VI': 'Virgin Islands',
        'VA': 'Virginia',
        'WA': 'Washington',
        'WV': 'West Virginia',
        'WI': 'Wisconsin',
        'WY': 'Wyoming'}
        if state_acroname in state:
            return state[state_acroname]
        else:
            return state_acroname
        
states_covid_daily['state']=states_covid_daily['state'].apply(clean_state_name)
covid_cumulative=states_covid_daily[['state','positive','hospitalizedCumulative','death','totalTestResults']]


Clean-population-data

In [3]:
df_population=pd.read_excel("population.xlsx",header=[1],skiprows=[0,1,-1])
df_population=df_population.rename(columns = {'Unnamed: 0':'state',2019:'2019population'})
df_population=df_population[['state', '2019population']].dropna()
def replacing(state):
    state=state.replace('.','')
    return state
df_population['state']=df_population['state'].apply(replacing)

df_population.head()

,state,2019population
0,United States,328239523.0
1,Northeast,55982803.0
2,Midwest,68329004.0
3,South,125580448.0
4,West,78347268.0


Merging

In [4]:
Covid_and_population=pd.merge(df_population,covid_cumulative,how='inner',on='state')
Covid_and_population['infection_rate']=Covid_and_population['positive']/Covid_and_population['2019population']
Covid_and_population['death_rate']=Covid_and_population['death']/Covid_and_population['positive']
Covid_and_population['test_rate']=Covid_and_population['totalTestResults']/Covid_and_population['2019population']
Covid_and_population['hospitalized_rate']=Covid_and_population['hospitalizedCumulative']/Covid_and_population['positive']
# Covid_and_population['hospitalized_rate'].fillna(value=np.(Covid_and_population['hospitalized_rate']), inplace=True)
Covid_and_population.head()

,state,2019population,positive,hospitalizedCumulative,death,totalTestResults,infection_rate,death_rate,test_rate,hospitalized_rate
0,Alabama,4903185.0,499819.0,45976.0,10148.0,2323788.0,0.101938,0.020303,0.473934,0.091985
1,Alaska,731545.0,56886.0,1293.0,305.0,1731628.0,0.077761,0.005362,2.367083,0.022730
2,Arizona,7278717.0,826454.0,57907.0,16328.0,7908105.0,0.113544,0.019757,1.086470,0.070067
3,Arkansas,3017804.0,324818.0,14926.0,5319.0,2736442.0,0.107634,0.016375,0.906766,0.045952
4,California,39512223.0,3501394.0,NaN,54124.0,49646014.0,0.088615,0.015458,1.256472,NaN


In [5]:
labels = pd.Categorical(['lowest25%','50%-75%','''25%-50%''','top25%'],ordered=True)

Covid_and_population['infection_rank'] = pd.qcut(Covid_and_population['infection_rate'], 4, labels = labels)
Covid_and_population['death_rank'] = pd.qcut(Covid_and_population['death_rate'], 4, labels=labels)
Covid_and_population['test_rank'] = pd.qcut(Covid_and_population['test_rate'], 4,labels=labels)

In [6]:
Covid_and_population.head()

,state,2019population,positive,hospitalizedCumulative,death,totalTestResults,infection_rate,death_rate,test_rate,hospitalized_rate,infection_rank,death_rank,test_rank
0,Alabama,4903185.0,499819.0,45976.0,10148.0,2323788.0,0.101938,0.020303,0.473934,0.091985,top25%,top25%,lowest25%
1,Alaska,731545.0,56886.0,1293.0,305.0,1731628.0,0.077761,0.005362,2.367083,0.022730,50%-75%,lowest25%,top25%
2,Arizona,7278717.0,826454.0,57907.0,16328.0,7908105.0,0.113544,0.019757,1.086470,0.070067,top25%,25%-50%,25%-50%
3,Arkansas,3017804.0,324818.0,14926.0,5319.0,2736442.0,0.107634,0.016375,0.906766,0.045952,top25%,50%-75%,50%-75%
4,California,39512223.0,3501394.0,NaN,54124.0,49646014.0,0.088615,0.015458,1.256472,NaN,50%-75%,50%-75%,25%-50%


In [10]:
def final_score(row):
    score = int()
    if row[-1]=='top25%':
        score += 100
    elif row[-1]=='25%-50%':
        score += 75
    elif row[-1]=='50%-75%':
        score += 50
    elif row[-1]=='lowest25%':
        score += 25
    if row[-2]=='top25%':
        score += 25
    elif row[-2]=='25%-50%':
        score += 50
    elif row[-2]=='50%-75%':
        score += 75
    elif row[-2]=='lowest25%':
        score += 100
    if  row[-3]=='top25%':
        score += 25
    elif row[-3]=='25%-50%':
        score += 50
    elif row[-3]=='50%-75%':
        score += 75
    elif row[-3]=='lowest25%':
        score += 100
    return score

Covid_and_population['final_score'] = Covid_and_population.apply(final_score, axis=1)
Covid_and_population['final_score']
Covid_and_population=Covid_and_population.sort_values(['final_score','infection_rate'],ascending=False)
Covid_and_population=Covid_and_population.reset_index().drop('index',axis=1)
Covid_and_population.index+=1
Covid_and_population

,state,2019population,positive,hospitalizedCumulative,death,totalTestResults,infection_rate,death_rate,test_rate,hospitalized_rate,infection_rank,death_rank,test_rank,final_score
1,Idaho,1787065.0,172931.0,7184.0,1879.0,645838.0,0.096768,0.010866,0.361396,0.041543,25%-50%,lowest25%,lowest25%,200
2,Utah,3205958.0,374850.0,14891.0,1976.0,2910121.0,0.116923,0.005271,0.907723,0.039725,top25%,lowest25%,50%-75%,175
3,Oklahoma,3956971.0,428997.0,24332.0,4534.0,3575297.0,0.108416,0.010569,0.903544,0.056718,top25%,lowest25%,50%-75%,175
4,Kansas,2913314.0,295861.0,9387.0,4812.0,1270547.0,0.101555,0.016264,0.436117,0.031728,top25%,50%-75%,lowest25%,175
5,Montana,1068778.0,100914.0,4630.0,1381.0,1104488.0,0.094420,0.013685,1.033412,0.045881,25%-50%,lowest25%,50%-75%,175
6,Texas,28995881.0,2686818.0,NaN,44451.0,19907384.0,0.092662,0.016544,0.686559,NaN,25%-50%,50%-75%,lowest25%,175
7,Kentucky,4467673.0,410709.0,19457.0,4819.0,3975672.0,0.091929,0.011733,0.889875,0.047374,25%-50%,lowest25%,50%-75%,175
8,North Carolina,10488084.0,872176.0,NaN,11502.0,9688838.0,0.083159,0.013188,0.923795,NaN,50%-75%,lowest25%,50%-75%,175
9,Virginia,8535519.0,585700.0,24661.0,9596.0,6038209.0,0.068619,0.016384,0.707421,0.042105,lowest25%,50%-75%,lowest25%,175
10,Washington,7614893.0,344532.0,19599.0,5041.0,5393756.0,0.045244,0.014631,0.708317,0.056886,lowest25%,50%-75%,lowest25%,175
